In [1]:
import os

from scipy.io import wavfile
from scipy import signal
import librosa as lr

import numpy as np
import pandas as pd

In [2]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected, flatten
from tflearn.layers.estimator import regression

/home/maria/Programs/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
from tqdm import tqdm 

In [4]:
DATASET = './datasets/tatoeba/'

import dataset_utils_misc
# dataset_utils_misc.create_dataset_lists(DATASET)

dataset_all = DATASET+'/dataset-txt/dataset-all.txt'
dataset_train = DATASET+'/dataset-txt/dataset-train.txt'
dataset_test = DATASET+'/dataset-txt/dataset-test.txt'
dataset_validation = DATASET+'/dataset-txt/dataset-validation.txt'

In [5]:
from misc_preloader import misc_preloader

X_train, y_train = misc_preloader(dataset_train, categorical_labels=True)
X_test, y_test = misc_preloader(dataset_test, categorical_labels=True)
X_val, y_val = misc_preloader(dataset_validation, categorical_labels=True)

In [6]:
X_train[31].shape

(192, 192)

In [8]:
n_classes = 6
for y in y_train:
    if not y.shape == (n_classes,):
        print ('Bad train sample')
        break

for y in y_val:
    if not y.shape == (n_classes,):
        print ('Bad validation sample')
        break
        
for y in y_test:
    if not y.shape == (n_classes,):
        print ('Bad test sample')
        break

In [9]:
input_shape = X_train[0].shape
# n_classes = y_train[0].shape[0]
LR = 0.001
filter_size = 3

In [10]:
def reshape_input(X):
    X = np.array(X).reshape([-1, input_shape[0], input_shape[1], 1])
    return X

pre_proc = tflearn.DataPreprocessing()
pre_proc.add_custom_preprocessing(reshape_input)

In [11]:
def create_model():
    
    tf.reset_default_graph()

    convnet = input_data(shape=[None, input_shape[0], input_shape[1], 1], name='input', 
                         data_preprocessing=pre_proc)

    convnet = conv_2d(convnet,16, filter_size, activation='relu')
    convnet = max_pool_2d(convnet, filter_size)
    
    convnet = conv_2d(convnet, 32, filter_size, activation='relu')
    convnet = max_pool_2d(convnet, filter_size)
    
    convnet = conv_2d(convnet, 64, filter_size, activation='relu')
    convnet = max_pool_2d(convnet, filter_size)
    
    convnet = conv_2d(convnet, 128, filter_size, activation='relu')
    convnet = max_pool_2d(convnet, filter_size)
    
    convnet = conv_2d(convnet, 256, filter_size, activation='relu')
    convnet = max_pool_2d(convnet, filter_size)    
    
    convnet = flatten(convnet)
    convnet = fully_connected(convnet, 512, activation='relu')
    convnet = dropout(convnet, 0.5)

    convnet = fully_connected(convnet, n_classes, activation='softmax')
    convnet = regression(convnet, optimizer='adam', learning_rate=LR, 
                         loss='categorical_crossentropy', name='targets')

    model = tflearn.DNN(convnet, tensorboard_dir='log')
    return model

In [12]:
model = create_model()

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [13]:
MODEL_NAME = 'models/cnn-misc-tatoeba.tflearn'
model.fit({'input': X_train}, {'targets': y_train}, n_epoch=10, 
          validation_set=({'input': X_val}, {'targets': y_val}), 
        snapshot_step=500, show_metric=True, run_id=MODEL_NAME, shuffle=True)

model.save(MODEL_NAME)    

Training Step: 1259  | total loss: 0.01363 | time: 1502.283s
| Adam | epoch: 010 | loss: 0.01363 - acc: 0.9954 -- iter: 8000/8056
Training Step: 1260  | total loss: 0.01286 | time: 1853.073s
| Adam | epoch: 010 | loss: 0.01286 - acc: 0.9959 | val_loss: 0.04578 - val_acc: 0.9891 -- iter: 8056/8056
--
INFO:tensorflow:/home/maria/Projects/DRU-W3-FP-6-Emotion-and-Tone-Analyzer/models/cnn-misc-tatoeba.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [14]:
model.evaluate(X_test, y_test)

[0.98212512531488871]

In [15]:
res = model.predict(X_test)

In [16]:
y_test_labels = [np.argmax(y_test[i]) for i in range (len(y_test))]
res_labels = [np.argmax(res[i]) for i in range (len(res))]
df = pd.DataFrame()
df['Pred'] = res_labels
df['Real'] = y_test_labels

categories = dataset_utils_misc.get_categories(DATASET)
category_unmapping = {}
for i in range(len(categories)):
    category_unmapping[i] = categories[i]
df.replace(category_unmapping)

,Pred,Real
0,eng,eng
1,fra,fra
2,spa,spa
3,rus,rus
4,rus,rus
5,rus,rus
6,spa,spa
7,spa,spa
8,eng,eng
9,fra,fra
